# Building A Spam Filter With Naive Bayes

In this project we are going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. 

To classify messages as spam, a computer does several things:

1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).


To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository.](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Explore the Dataset

Lets start by examining the dataset.

In [1]:
import pandas as pd
sms = pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['Label','SMS'])
print('Size of Dataset: \nRows: ',sms.shape[0],'\nColumns: ',sms.shape[1])

sms.head(5)

display(sms['Label'].value_counts(normalize=True)*100)

Size of Dataset: 
Rows:  5572 
Columns:  2


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

This dataset has 5572 sms messages with 2 columns: label and sms. the Label is 'ham' if the message is not spam, and 'spam' if spam. The SMS column shows the message that is classified. 

86.59% of all messages are classified as 'ham', and 13.4% of messages are classified as spam.

Now that we've become a bit familiar with the dataset, we can move on to building the spam filter.

## Training and Test Set

We're now going to split our dataset into a training set and test set. We will use the training set to "train" the computer how to classify messages, and the test set to test how good the spam filter performs.

We will keep 80% of our dataset for training, and 20% for testing.

In [2]:
# Randomize the dataset
sms_randomized = sms.sample(frac=1,random_state =1)

# Calculate index for split
training_test_index = round(len(sms_randomized) * 0.8)

# Training/Test split
train = sms_randomized[:training_test_index].reset_index(drop=True)
test = sms_randomized[training_test_index:].reset_index(drop=True)

print("Training dataset percentage of spam messages: \n",train['Label'].value_counts(normalize=True)*100)
print("Test dataset percentage of spam messages: \n",test['Label'].value_counts(normalize=True)*100)

Training dataset percentage of spam messages: 
 ham     86.54105
spam    13.45895
Name: Label, dtype: float64
Test dataset percentage of spam messages: 
 ham     86.804309
spam    13.195691
Name: Label, dtype: float64


In the overall dataset, 87% of messages were ham and 13% spam. After splitting our dataset into training and test datasets, we see both datasets have a similar percentage breakdown of 87%/13% ham/spam, so these datasets are representative of the larger dataset.

We'll now move on to cleaning the dataset.

## Data Cleaning

To calculate the probabilities needed for the Naive Bayes algorithm, we need to perform some data cleaning to bring the data in a format that will allow us to extract the information we need from these messages.

We want to transform the SMS column to a series of new columns, where each column represents a unique word from the vocabulary and the occurence of each unique word in each message.

We will also convert all the messages to lower case and remove punctionation

### Letter Case and Punctuation

In [3]:
#Before cleaning
train.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [4]:
#Removing Punctuation and letter case from messages

train['SMS'] = train['SMS'].str.replace('\W',' ').str.lower()

#After cleaning
train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary List

Let's now create the vocabulary list, which is a list with all the unique words in our training set.

In [5]:
vocabulary = []

train['SMS'] = train['SMS'].str.split()

for sms in train['SMS']:
    for word in sms:
        if word not in vocabulary:
            vocabulary.append(word)

vocabulary = set(vocabulary)
vocabulary = list(vocabulary)
vocabulary[:5]

['ads', 'stressfull', 'gay', 'extra', 'thanx4']

### Creating the final DataFrame

We're now going to take the vocabulary list we just created to make the transformation we want: where each word in the vocab list is a column and the value for that column is the frequency at which the word appears in an SMS message. 

In [6]:
word_counts_per_sms = {unique_word: [0]*len(train['SMS']) for unique_word in vocabulary}

for index,sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

word_counts = pd.DataFrame.from_dict(word_counts_per_sms) 

train_clean = pd.concat([train,word_counts],axis=1)
train_clean.head()

,Label,SMS,ads,stressfull,gay,extra,thanx4,bowl,per,takecare,...,drinks,fringe,idew,epi,thing,kolathupalayam,two,jess,places,thank
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

We have now finished cleaning the training data, so now we can begin creating the spam filter. The Naive Bayes algorithm will need to know the probability values of the 2 equations below to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- NSpam, NHam, NVocabulary

We will use Laplace smoothing and set alpha = 1

In [7]:
#Isolate spam and ham messages into separate datasets
spam_messages = train_clean[train_clean['Label'] == 'spam']
ham_messages = train_clean[train_clean['Label'] == 'ham']

#Calculate P(spam) and P(ham)
p_spam = len(spam_messages)/len(train_clean)
p_ham = len(ham_messages)/len(train_clean)

#N_spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

#N_ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

#n_vocabuary
n_vocabulary = len(vocabulary)

#Laplace smoothing variable
alpha = 1

## Calculating Parameters

Now that the constant terms have been calculated, we move on to calculating the paramters P(wi|spam) and P(wi|ham). Each paramter will be a conditional probability value associated with each word in the vocab

The parameters are calculated using these formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [8]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying A New Message

Now that we have our parameters calculated, we can start creating a spam filter. We need to create a function that:

- Takes in as input a new message (w1, w2, ..., wn).
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [9]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]


    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
        


In [10]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [11]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy with Test Dataset

The two messages we tested above look like the filter classifies messages properly. Let's see how well the filter performs on our test set, a dataset of 1,115 messages. 

We need to rewrite our classify function to return classification labels rather than printing them. 

We will then apply this function to our test set to create a new column.

In [12]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [13]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Next, we will create calculate the accuracy of this filter by checking the number of correctly classified messages

In [14]:
correct = 0
total = len(test)

for index,row in test.iterrows():
    label = row['Label']
    predicted = row['predicted']
    
    if predicted == label:
        correct += 1

accuracy = round(correct/total,ndigits = 3)

print("Accuracy of Spam Filter = ", accuracy*100,'%')

Accuracy of Spam Filter =  98.7 %


The accuracy is roughly 98.7%, which is much higher than our goal of 80% accuracy. This means for our test set, for the 1,115 messages, 1101 messages were classfied correctly.

## Next Steps

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.

Next steps to improve this filter include:
- Analyze the 14 messages that were classified incorrectly and try to figure out why they were classified incorrectly by the algorithm
- Make the filtering process sensitive to letter case to improve the algorithm